In [124]:
from copy import copy
import os
import shutil
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns
import cost_function
from database import CityLearnDatabase, SQLiteDatabase
from utilities import split_lines

In [125]:
formatted_name = {
    'marlisa':{True:r'$\mathrm{MARLISA_{RBC_{Basic}}}$',False:r'$\mathrm{MARLISA_{RBC_{Optimized}}}$'},
    'sac':{True:r'$\mathrm{SAC_{RBC_{Basic}}}$',False:r'$\mathrm{SAC_{RBC_{Optimized}}}$'},
    'basic_rbc':{True:r'$\mathrm{RBC_{Basic}}$',False:r'$\mathrm{RBC_{Basic}}$'},
    'optimized_rbc':{True:r'$\mathrm{RBC_{Optimized}}$',False:r'$\mathrm{RBC_{Optimized}}$'},
}
seasons = {
    'spring':(3,4,5),
    'summer':(6,7,8),
    'fall':(9,10,11),
    'winter':(12,1,2),
}
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0

# Reward Function Design
***
$E$ is an array of the district buildings' electricity demand, $n$ is the number of buildings in the district and $t$ is the current timestep.

1. __Default__
    $$
    r(t)^\textrm{def} = E(t)
    $$

2. __MARLISA__
    $$
    r(t)^\textrm{marl} = \textrm{sign}\left(r(t)^\textrm{def}\right) \times 0.01 \times \left|{r(t)^\textrm{def}}\right|^2 \times \textrm{max}\left(0, -\sum_{i=1}^{n-1}{r(t)^\textrm{def}_i}\right)
    $$

3. __SAC__
    $$
    r(t)^\textrm{sac} = \left[\textrm{min}\left(0,{r(t)_0^\textrm{def}}^3\right), \dots, \textrm{min}\left(0,{r(t)_{n-1}^\textrm{def}}^3\right)\right]
    $$

4. __Ramping Square__
    $$
    r(t)^\textrm{ramp} = \left[-\left(\left(\sum_{i=0}^{n-1}{r(t)^\textrm{def}_i} - \sum_{i=0}^{n-1}{r(t-1)^\textrm{def}_i}\right)^2\right)_0, \dots, -\left(\left(\sum_{i=0}^{n-1}{r(t)^\textrm{def}_i} - \sum_{i=0}^{n-1}{r(t-1)^\textrm{def}_i}\right)^2\right)_{n-1}\right] \div n
    $$

5. __Exponential__
    $$
    r(t)^\textrm{exp} = \left[\left(-e^{0.0002 \times \sum_{i=0}^{n-1}{-r(t)^\textrm{def}_i}}\right)_0, \dots, \left(-e^{0.0002 \times \sum_{i=0}^{n-1}{-r(t)^\textrm{def}_i}}\right)_{n-1}\right] \div n
    $$

    Where 0.0002 is a scaling factor to avoid high exponents defined as $\frac{\lambda}{50}$. We use $\lambda = 0.01$.

6. __Mixed__
    $$
    r(t)^\textrm{mix} = r(t)^\textrm{ramp} + r(t)^\textrm{exp}
    $$

# Simulation Analysis
***

### C8 
- training off-line from the fixed logs of an external behavior policy.The benchmark here is to learn a control law  from data generated by a sub-optimal reference controller, .e.g., a rule based controller, which is often available, essentially a system log. In addition to the control environment, datasets of various size, e.g.,two weeks, one months, 6 months should be provided that are generated with a known reference rule based controller. Then, the controllers can be evaluated on the ability to improve these baselines.

In [3]:
shared_directory = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/lab_computer/simulation_output_c8'
c8_database_filepath = os.path.join(shared_directory,'simulation.db')
# source_filepaths = [os.path.join(shared_directory,f'simulation_{i}.db') for i in range(1,41)]
# CityLearnDatabase.concatenate(c8_database_filepath,source_filepaths)
database = SQLiteDatabase(c8_database_filepath)

In [4]:
# metadata
metadata = database.query_table("""
SELECT
    s.id AS simulation_id,
    s.name AS simulation_name,
    e.id AS environment_id,
    a.name AS agent_name,
    AVG(a.seed) AS seed,
    AVG(a.basic_rbc) AS basic_rbc,
    AVG(a.exploration_period) AS exploration_period
FROM agent a
LEFT JOIN building b ON b.agent_id = a.id
LEFT JOIN environment e ON e.id = b.environment_id
LEFT JOIN simulation s ON s.id = e.simulation_id
GROUP BY
    s.id,
    e.id,
    a.name
""")
group_list = []

for i, (name, group) in enumerate(metadata.groupby(['agent_name','basic_rbc','exploration_period'])):
    group = group.copy()
    group.loc[:,'simulation_set'] = i
    group_list.append(group)

metadata = pd.concat(group_list,ignore_index=True)
metadata = metadata.sort_values(['simulation_set']).reset_index(drop=True)
display(metadata.head())

# storage capacity
query = """
SELECT
    b.id,
    0 AS "Cooling Storage (h)",
    0 AS "DHW Storage (h)",
    e.capacity AS "Electrical Storage (kW)",
    b.solar_power_installed AS "PV (kW)"
FROM building b
LEFT JOIN cooling_storage c ON c.building_id = b.id
LEFT JOIN dhw_storage d ON d.building_id = b.id
LEFT JOIN electrical_storage e ON e.building_id = b.id
WHERE environment_id = 1
"""
storage_data = database.query_table(query)
storage_data = storage_data.set_index('id')
storage_data = storage_data.round(0).astype(int)

,simulation_id,simulation_name,environment_id,agent_name,seed,basic_rbc,exploration_period,simulation_set
0,37,simulation_37,37,basic_rbc,0.0,1.0,0.0,0
1,38,simulation_38,38,basic_rbc,0.0,1.0,0.0,0
2,14,simulation_14,14,marlisa,0.0,0.0,744.0,1
3,16,simulation_16,16,marlisa,1.0,0.0,744.0,1
4,18,simulation_18,18,marlisa,2.0,0.0,744.0,1


In [5]:
# total net electricity consumption
data_list = []

for (simulation_set, agent_name, basic_rbc, exploration_period), group in metadata.groupby(['simulation_set','agent_name','basic_rbc','exploration_period']):
    environment_ids = tuple(group['environment_id'].tolist())
    query = f"""
    SELECT
        SUM(value) AS value
    FROM (
        SELECT
            t.timestep,
            AVG(e.net_electric_consumption) AS value
        FROM environment_timeseries e
        LEFT JOIN timestep t ON t.id = e.timestep_id
        WHERE e.environment_id IN {environment_ids}
        GROUP BY
            t.timestep
    )
    """
    plot_data = database.query_table(query)
    plot_data['agent_name'] = agent_name
    plot_data['exploration_period'] = exploration_period
    plot_data['basic_rbc'] = basic_rbc
    data_list.append(plot_data)

plot_data = pd.concat(data_list,ignore_index=True)
bar_plot_data = plot_data[~plot_data['agent_name'].isin(['basic_rbc','optimized_rbc'])].copy()
fig, ax = plt.subplots(1,1,figsize=(3,2))
x_ticks = np.arange(bar_plot_data.groupby(['agent_name','basic_rbc']).size().shape[0])
width = 0.3

for i, (exploration_period, group) in enumerate(bar_plot_data.groupby('exploration_period')):
    x = [(x_tick - width*len(group)/2) + width*(i + 0.5) for x_tick in x_ticks]
    y = group['value'].tolist()
    ax.barh(x,y,width,label=int(exploration_period))

ax.set_xlabel('kWh')
ax.set_yticks(x_ticks)
y_tick_labels = [
    formatted_name[agent_name][basic_rbc] 
    for agent_name, basic_rbc in bar_plot_data.groupby(
        ['agent_name','basic_rbc',]
    ).size().reset_index()[['agent_name','basic_rbc']].to_records(index=False)
]
ax.set_yticklabels(y_tick_labels)
ax.tick_params(axis='y',rotation=0)
ax.margins(0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

fig.axes[0].legend(
    title='Offline period',
    bbox_to_anchor=(1.5,1),
    loc='upper right', 
    framealpha=0,
    ncol=1
)
plt.savefig('figures/total_net_electricity_consumption.pdf',bbox_inches='tight',transparent=True)
plt.close()

In [86]:
# average weekly x electric consumption
columns = ['net_electric_consumption','electric_consumption_electric_storage','electric_consumption_cooling_storage','electric_consumption_dhw_storage']
plot_metadata = metadata[metadata['agent_name'].isin(['marlisa','sac'])].copy()
end_timestep = database.query_table('SELECT MAX(timestep) FROM timestep').iloc[0,0]
start_timestep = plot_metadata['exploration_period'].max() + 1
row_count = plot_metadata.groupby(['agent_name','basic_rbc']).size().shape[0]
column_count = len(seasons)
figsize = (3.2*column_count,1.5*row_count)

for column in columns:
    fig, axs = plt.subplots(row_count,column_count,figsize=figsize,sharey=True,sharex=True)

    for i, season in enumerate(seasons):
        for j, ((agent_name, basic_rbc), agent_group) in enumerate(plot_metadata.groupby(['agent_name','basic_rbc'])):

            for exploration_period, group in agent_group.groupby(['exploration_period']):
                environment_ids = tuple(group['environment_id'].tolist())
                query = f"""
                SELECT
                    t.day_type,
                    t.hour,
                    AVG(e.{column}) AS value
                FROM environment_timeseries e
                LEFT JOIN timestep t ON t.id = e.timestep_id
                WHERE t.timestep BETWEEN {start_timestep} AND {end_timestep}
                AND e.environment_id IN {environment_ids}
                AND t.month IN {seasons[season]}
                GROUP BY
                    t.day_type,
                    t.hour
                """
                plot_data = database.query_table(query)
                x, y = plot_data.index, plot_data['value']
                axs[j,i].plot(x,y,label=int(exploration_period))

            if j == 0:
                axs[j,i].set_title(f'{season.capitalize()}')
            else:
                axs[j,i].set_title(None)

            if i == 0:
                axs[j,i].set_ylabel(f'{formatted_name[agent_name][basic_rbc]}\n{"kWh"}')
            else:
                axs[j,i].set_ylabel(None)
            
            axs[j,i].xaxis.set_major_locator(ticker.MultipleLocator(base=24))
            axs[j,i].spines['top'].set_visible(False)
            axs[j,i].spines['right'].set_visible(False)

            if column != 'net_electric_consumption':
                axs[j,i].spines['bottom'].set_position('zero')
            else:
                pass

    axs[j,i].legend(
        title='Offline period',ncol=len(agent_group['exploration_period'].unique()),
        bbox_to_anchor=(0.5,0.09),bbox_transform=fig.transFigure,loc='upper center',framealpha=0,
    )
    fig.align_ylabels()
    plt.savefig(f'figures/average_weekly_{column}.pdf',bbox_inches='tight',facecolor='white')
    plt.close()

In [148]:
# seasonal net electric consumption
columns = ['net_electric_consumption',]
plot_metadata = metadata[metadata['agent_name'].isin(['marlisa','sac'])].copy()
end_timestep = database.query_table('SELECT MAX(timestep) FROM timestep').iloc[0,0]
row_count = plot_metadata.groupby(['agent_name','basic_rbc']).size().shape[0]
years = [2014,2015]
first_year_start_timestep = 4344
days = 7
column_count = len(years)
figsize = (6*column_count,1.5*row_count)
fig, axs = plt.subplots(row_count,column_count,figsize=figsize,sharey=True,sharex='col')

for i, year in enumerate(years):
    start_timestep = i*8760 + first_year_start_timestep
    end_timestep = start_timestep + days*24

    for j, ((agent_name, basic_rbc), agent_group) in enumerate(plot_metadata.groupby(['agent_name','basic_rbc'])):
        plotted_flexibility = False

        for exploration_period, group in agent_group.groupby(['exploration_period']):
            environment_ids = tuple(group['environment_id'].tolist())
            query = f"""
            SELECT
                t.timestep,
                AVG(e.net_electric_consumption) AS net_electric_consumption,
                AVG(e.net_electric_consumption + e.electric_generation - (
                    e.electric_consumption_electric_storage 
                    + e.electric_consumption_electric_storage 
                    + e.electric_consumption_electric_storage
                )) AS electric_consumption_no_pv_no_storage,
                AVG(e.net_electric_consumption - (
                    e.electric_consumption_electric_storage 
                    + e.electric_consumption_electric_storage 
                    + e.electric_consumption_electric_storage
                )) AS electric_consumption_no_storage
            FROM environment_timeseries e
            LEFT JOIN timestep t ON t.id = e.timestep_id
            WHERE t.timestep BETWEEN {start_timestep} AND {end_timestep}
            AND e.environment_id IN {environment_ids}
            GROUP BY
                t.timestep
            """
            plot_data = database.query_table(query).iloc[0:int(days*24)]
            x, y = plot_data['timestep'], plot_data['net_electric_consumption']

            if not plotted_flexibility:
                axs[j,i].plot(x,plot_data['electric_consumption_no_pv_no_storage'],color='black',linestyle='--',label='No PV & storage')
                # axs[j,i].plot(x,plot_data['electric_consumption_no_storage'],color='red',linestyle='--',label='No storage')
                plotted_flexibility = True
            else:
                pass

            axs[j,i].plot(x,y,label=int(exploration_period))

        axs[j,i].set_ylabel(f'{formatted_name[agent_name][basic_rbc]}\n{"kWh"}')
        axs[j,i].xaxis.set_major_locator(ticker.MultipleLocator(base=48))
        axs[j,i].spines['top'].set_visible(False)
        axs[j,i].spines['right'].set_visible(False)

        if i == 0:
            axs[j,i].set_ylabel(formatted_name[agent_name][basic_rbc])
        else:
            axs[j,i].set_ylabel(None)

        if j == 0:
            axs[j,i].set_title(int(year))
        else:
            axs[j,i].set_title(None)

axs[j,i].legend(ncol=6,bbox_to_anchor=(0.5,0.09),bbox_transform=fig.transFigure,loc='upper center',framealpha=0,)
fig.align_ylabels()
plt.savefig(f'figures/net_electric_consumption_snapshot.pdf',bbox_inches='tight',transparent=True)
plt.close()

In [7]:
# annual net electric consumption
fig, ax = plt.subplots(1,1,figsize=(6,3))

for (simulation_set, exploration_period, agent_name, basic_rbc), group in metadata.groupby(['simulation_set','exploration_period','agent_name', 'basic_rbc']):
    environment_ids = tuple(group['environment_id'].tolist())
    query = f"""
    SELECT
        CAST (timestep / 8760 AS INTEGER) + 1 AS year,
        SUM(value) AS value
    FROM (
        SELECT
            t.timestep,
            AVG(e.net_electric_consumption) AS value
        FROM environment_timeseries e
        LEFT JOIN timestep t ON t.id = e.timestep_id
        WHERE e.environment_id IN {environment_ids}
        GROUP BY
            t.timestep
    )
    GROUP BY
        CAST (timestep / 8760 AS INTEGER) + 1
    """
    plot_data = database.query_table(query)
    x = plot_data['year']
    y = plot_data['value']
    ax.plot(x,y,label=f'{formatted_name[agent_name][basic_rbc]} - {int(exploration_period)}')
    ax.set_xticks(range(int(min(x)),int(max(x)+1)))
    ax.set_xlabel('Year')
    ax.set_ylabel('Net electric consumption (kWh)')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

ax.legend(loc=4,bbox_to_anchor=(1.55,0))
plt.savefig(f'figures/annual_net_electric_consumption.pdf',bbox_inches='tight',facecolor='white')
plt.close()

In [146]:
# cost functions
plot_data_list = []

for i, ((simulation_set, exploration_period, agent_name, basic_rbc), group) in enumerate(metadata.groupby(['simulation_set','exploration_period','agent_name', 'basic_rbc'])):
    environment_ids = tuple(group['environment_id'].tolist())
    query = query = f"""
    SELECT
        t.timestep,
        AVG(e.net_electric_consumption) AS net_electric_consumption
    FROM environment_timeseries e
    LEFT JOIN timestep t ON t.id = e.timestep_id
    WHERE e.environment_id IN {environment_ids}
    GROUP BY
        t.timestep
    """
    plot_data = database.query_table(query)
    
    # cost functions
    plot_data['ramping_cost_function'] = cost_function.ramping(plot_data['net_electric_consumption'])
    plot_data['load_factor_cost_function'] = cost_function.load_factor(plot_data['net_electric_consumption'])
    plot_data['average_daily_peak_cost_function'] = cost_function.average_daily_peak(plot_data['net_electric_consumption'])
    # plot_data['annual_peak_demand_cost_function'] = cost_function.peak_demand(plot_data['net_electric_consumption'])
    plot_data['net_electric_consumption_cost_function'] = cost_function.net_electric_consumption(plot_data['net_electric_consumption'])
    plot_data['agent_name'] = agent_name
    plot_data['exploration_period'] = exploration_period
    plot_data['basic_rbc'] = basic_rbc
    plot_data['simulation_set'] = simulation_set
    plot_data_list.append(plot_data)

# calculate normalized cost
plot_data = pd.concat(plot_data_list,ignore_index=True)
plot_data = plot_data.drop(columns=['net_electric_consumption'])
cost_function_columns = [c for c in plot_data.columns if c.endswith('cost_function')]
id_vars = [c for c in plot_data.columns if c not in cost_function_columns]
plot_data = plot_data.melt(id_vars=id_vars,value_vars=cost_function_columns)
plot_data['variable'] += '_norm'
rbc_data = plot_data[plot_data['agent_name'].isin(['basic_rbc','optimized_rbc'])].copy()
rbc_data['basic_rbc'] = False
rbc_data.loc[rbc_data['agent_name']=='basic_rbc','basic_rbc'] = True
rbc_data = rbc_data.rename(columns={'value':'baseline_value'})
rbc_data = rbc_data[['timestep','variable','baseline_value','basic_rbc']].copy()
plot_data = plot_data.drop(rbc_data.index)
plot_data = pd.merge(plot_data,rbc_data,on=['timestep','variable','basic_rbc'],how='left')
plot_data['norm_value'] = plot_data['value']/plot_data['baseline_value']

# plot normalized cost functions
column_count = plot_data.groupby(['agent_name','basic_rbc']).size().shape[0]
row_count = len(cost_function_columns)
figsize = (3*column_count,1.25*row_count)
fig, axs = plt.subplots(row_count,column_count,figsize=figsize,sharey='row',sharex=True)

for i, ((agent_name, basic_rbc), group) in enumerate(plot_data.groupby(['agent_name','basic_rbc'])):
    for j, (variable, var_group) in enumerate(group.groupby(['variable'])):
        for exploration_period, env_group in var_group.groupby(['exploration_period']):
            x, y = env_group['timestep'], env_group['norm_value']
            axs[j,i].plot(x,y,label=int(exploration_period))

        axs[j,i].axhline(1,color='black',linestyle='--',)
        axs[j,i].spines['top'].set_visible(False)
        axs[j,i].spines['right'].set_visible(False)
        axs[j,i].xaxis.set_major_locator(ticker.MultipleLocator(8760))
        axs[j,i].legend().set_visible(False)

        if i == 0:
            ylabel = variable.replace('_cost_function_norm','').replace('_',' ').capitalize()
            ylabel = split_lines(ylabel,12)
            axs[j,i].set_ylabel(ylabel)
        else:
            axs[j,i].set_ylabel(None)

        if j == 0:
            axs[j,i].set_title(formatted_name[agent_name][basic_rbc])
        else:
            axs[j,i].set_title(None)

axs[j,i].legend(
    title='Offline period',ncol=len(plot_data['exploration_period'].unique()),
    bbox_to_anchor=(0.5,0.09),bbox_transform=fig.transFigure,loc='upper center',framealpha=0,
)
fig.align_ylabels()
plt.savefig('figures/normalized_cost_function.pdf',bbox_inches='tight',transparent=True)
plt.close()

In [72]:
# average weekly SOC for different seasons in last year
storage_timeseries_tables = ['cooling_storage_timeseries','dhw_storage_timeseries','electrical_storage_timeseries']
storage_tables = ['cooling_storage','dhw_storage','electrical_storage']
plot_metadata = metadata[metadata['agent_name'].isin(['marlisa','sac'])].copy()
end_timestep = database.query_table('SELECT MAX(timestep) FROM timestep').iloc[0,0]
start_timestep = plot_metadata['exploration_period'].max() + 1
row_count = plot_metadata.groupby(['agent_name','basic_rbc']).size().shape[0]
column_count = len(seasons)
figsize = (3.2*column_count,1.5*row_count)

for storage_table, storage_timeseries_table in zip(storage_tables,storage_timeseries_tables):
    fig, axs = plt.subplots(row_count,column_count,figsize=figsize,sharey=True,sharex=True)

    for i, season in enumerate(seasons):
        for j, ((agent_name, basic_rbc), agent_group) in enumerate(plot_metadata.groupby(['agent_name','basic_rbc'])):
            for exploration_period, group in agent_group.groupby(['exploration_period']):
                environment_ids = tuple(group['environment_id'].tolist())
                query = f"""
                SELECT
                    t.day_type,
                    t.hour,
                    AVG(st.soc/sm.capacity) AS soc
                FROM {storage_timeseries_table} st
                LEFT JOIN timestep t ON t.id = st.timestep_id
                LEFT JOIN {storage_table} sm ON sm.id = st.{storage_table}_id
                LEFT JOIN building b ON b.id = sm.building_id
                LEFT JOIN environment e ON e.id = b.environment_id
                WHERE t.timestep BETWEEN {start_timestep} AND {end_timestep}
                AND e.id IN {environment_ids}
                AND t.month IN {seasons[season]}
                GROUP BY
                    t.day_type,
                    t.hour
                ORDER BY
                    t.day_type ASC,
                    t.hour ASC
                """
                plot_data = database.query_table(query)
                x, y = plot_data.index, plot_data['soc']
                axs[j,i].plot(x,y,label=int(exploration_period))

            axs[j,i].spines['top'].set_visible(False)
            axs[j,i].spines['right'].set_visible(False)
            axs[j,i].xaxis.set_major_locator(ticker.MultipleLocator(24))
            axs[j,i].legend().set_visible(False)

            if i == 0:
                axs[j,i].set_ylabel(formatted_name[agent_name][basic_rbc])
            else:
                axs[j,i].set_ylabel(None)

            if j == 0:
                axs[j,i].set_title(season.capitalize())
            else:
                axs[j,i].set_title(None)

    axs[j,i].legend(
        title='Offline period',ncol=len(plot_metadata['exploration_period'].unique()),
        bbox_to_anchor=(0.5,0.09),bbox_transform=fig.transFigure,loc='upper center',framealpha=0,
    )
    fig.align_ylabels()
    plt.savefig(f'figures/{storage_table}_soc_weekly_average.pdf',bbox_inches='tight',transparent=True)
    plt.close()

In [ ]:
# end_timestep = database.query_table('SELECT MAX(timestep) FROM timestep').iloc[0,0]
# start_timestep = end_timestep - 8758
# storage_timeseries_tables = ['cooling_storage_timeseries','dhw_storage_timeseries','electrical_storage_timeseries']
# storage_tables = ['cooling_storage','dhw_storage','electrical_storage']
# plot_metadata = metadata[metadata['agent_name'].isin(['marlisa','sac'])].copy()
# row_count = len(plot_metadata['simulation_set'].unique())
# column_count = 1
# figsize=(18,row_count*4)
# shading = 'nearest'

# for storage_table, storage_timeseries_table in zip(storage_tables,storage_timeseries_tables):
#     fig, axs = plt.subplots(row_count,column_count,figsize=figsize)
#     for ax, ((simulation_set, agent_name, basic_rbc, exploration_period), group) in zip(fig.axes, plot_metadata.groupby(['simulation_set','agent_name','basic_rbc','exploration_period'])):
#         environment_ids = tuple(group['environment_id'].tolist())
#         query = f"""
#         SELECT
#             timestep,
#             day,
#             hour,
#             AVG(soc) AS soc
#         FROM (
#             SELECT
#                 t.timestep,
#                 CAST (t.timestep/24 AS TYPE INTEGER) + 1 AS day,
#                 t.timestep - CAST (t.timestep/24 AS TYPE INTEGER)*24 AS hour,
#                 st.soc/sm.capacity AS soc
#             FROM {storage_timeseries_table} st
#             LEFT JOIN timestep t ON t.id = st.timestep_id
#             LEFT JOIN {storage_table} sm ON sm.id = st.{storage_table}_id
#             LEFT JOIN building b ON b.id = sm.building_id
#             LEFT JOIN environment e ON e.id = b.environment_id
#             WHERE t.timestep BETWEEN {start_timestep} AND {end_timestep}
#             AND e.id IN {environment_ids}
#         )
#         GROUP BY
#             day,
#             hour,
#             timestep
#         """
#         plot_data = database.query_table(query)
#         plot_data = plot_data.pivot(index='hour',columns='day',values='soc')
#         x, y, z = plot_data.columns.tolist(), plot_data.index.tolist(), plot_data.values
#         divnorm = colors.TwoSlopeNorm(vcenter=0)
#         cmap = copy(cm.get_cmap('Blues'))
#         cmap.set_bad(color='grey')
#         pcm = ax.pcolormesh(x,y,z,shading=shading,cmap=cmap,norm=divnorm)
#         ax.set_xlabel('Day')
#         ax.set_ylabel('Hour')
#         agent_name = agent_name if agent_name != 'marlisa' else ('basic_rbc + marlisa' if basic_rbc else 'optimized_rbc + marlisa')
#         ax.set_title(f'{agent_name} - {exploration_period}')
#         cb = fig.colorbar(pcm,ax=ax,fraction=0.1,pad=0.02,label='SOC')
    
#     fig.suptitle(f'{storage_table}',y=1.0,fontsize=16)
#     plt.tight_layout()
#     plt.savefig(f'figures/{storage_table}_soc_last_year.png',bbox_inches='tight',facecolor='white')
#     plt.close()